## k-nearest-neighbor algorithm in plain Python

The k-nn algorithm is a simple **supervised** machine learning algorithm that can be used both for classification and regression. It's an **instance-based** algorithm. So instead of estimating a model, it stores all training examples in memory and makes predictions using a similarity measure. 

But it requires a large amount of memory, which depends directly on the number of instances. And it needs to be compared with known instances when predicting, so the running time will be relatively longer.

Given an input example, the k-nn algorithm retrieves the k most similar instances from memory. Similarity is defined in terms of distance, that is, the training examples with the smallest (Euclidean, Manhattan, ect.) distance to the input example are considered to be most similar.

The target value of the input example is computed as follows:  
  
Classification:  
a) unweighted: output the most common classification among the k-nearest neighbors  
b) weighted: sum up the weights of the k-nearest neighbors for each classification value, output classification with highest weight  
  
Regression:  
a) unweighted: output the average of the values of the k-nearest neighbors  
b) weighted: for all classification values,  sum up classification value$*$weight and divide the result trough the sum of all weights  

The weighted k-nn version is a refined version of the algorithm in which the contribution of each neighbor is *weighted* according to its distance to the query point. Below, we implement the basic unweighted version of the k-nn algorithm for the digits dataset from sklearn.

# Example 3.1 in Lihang's book

In [1]:
import numpy as np

In [2]:
x1 = np.array([1,1], dtype = np.float32)
x2 = np.array([5,1], dtype = np.float32)
x3 = np.array([4,4], dtype = np.float32)

In [3]:
# distance between x and y, default is Euclidean distance
def Dist(x, y, p = 2):
    dist = np.linalg.norm(x - y, p)
    return dist

In [4]:
for i in range(1, 5):
    dist_x1_x2 = Dist(x1, x2, p = i)
    print('L{}(x1, x2) = {:.3}'.format(i, dist_x1_x2))

L1(x1, x2) = 4.0
L2(x1, x2) = 4.0
L3(x1, x2) = 4.0
L4(x1, x2) = 4.0


In [5]:
for i in range(1, 5):
    dist_x1_x3 = Dist(x1, x3, p =i)
    print('L{}(x1, x3) = {:.3}'.format(i, dist_x1_x3))

L1(x1, x3) = 6.0
L2(x1, x3) = 4.24
L3(x1, x3) = 3.78
L4(x1, x3) = 3.57


Example 3.1 illustrates that the vector $\boldsymbol x_2$ or the vector $\boldsymbol x_3$ is closest to the vector $\boldsymbol x_1$ if $p <= 2$ or $p >= 3$ respectively.   

# KNN Algorithm

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [7]:
iris = load_iris()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)

In [9]:
clf_sk = KNeighborsClassifier(n_neighbors=3)
clf_sk.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [10]:
clf_sk.score(X_test, y_test)

0.9666666666666667

In [11]:
y_predict = clf_sk.predict(X_test)

In [12]:
y_predict

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0])

In [13]:
y_test

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0])